In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [3]:
from gensim.models import KeyedVectors
w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
# w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[339] 2022-01-16 10:47:59,779 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[339] 2022-01-16 10:49:03,010 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-01-16T10:49:03.006364', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}


In [4]:
class PNModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(PNModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        # self.fb_dim = 4
        # self.fb_dim = 0
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        # self.hidden2tag = nn.Linear(self.hid2+self.fb_dim, tagset_size)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        # fb = x[:, :self.fb_dim]
        y = F.relu(self.fc1(x))
        # y = F.relu(self.fc1(x[]))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [5]:
model_path = "../models/social/"
model_name = "pn_dnn_v1.pickle"
modelM = DataManager(model_path)
PNmodel = modelM.load_data(model_name)

success load : ../models/social/pn_dnn_v1.pickle


In [6]:
def is_ADJ(token):
    if token.pos_ == "ADJ":
        return True
    return False

In [7]:
def dependent_ADJ_NOUN(text):
    doc = nlp(text)
    adj_nouns = []
    for token in doc:
        if is_ADJ(token):
            # print(token)
            for child in token.children:
                if "NOUN" == child.pos_:
                    # print(child, token)
                    adj_nouns.append([child.lemma_, token.lemma_])
    return adj_nouns

In [8]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [9]:
# error = "Lack of common sense"
error = "Lack of sociality"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [10]:
y.count(1)

7

In [11]:
def w2v_list(words, w2v_model):
    vector = []
    for w in words:
        if w not in w2v_model:
            return []
        else:
            vector.append(w2v_model[w] )
    return vector

In [12]:
y_pred = []
for utt in tqdm(sys_utt):
    adj_nouns = dependent_ADJ_NOUN(utt)
    if len(adj_nouns)==0:
        y_pred.append(0)
        continue
    # 形容詞と名詞のペアが存在する
    is_lack_common = False
    for pair in adj_nouns:
        an_vector = w2v_list(pair, w2v_model)

        if len(an_vector)==0:
            # y_pred.append(0)
            continue
        
        with torch.no_grad():
            an_tensor = torch.tensor(an_vector, device='cuda:0').float()
            pred = np.array(PNmodel(an_tensor).cpu()).argmax(axis=1)
        del an_tensor
        noun_pn = pred[0]
        adj_pn = pred[1]

        # ネガティブな名詞をポジティブに形容
        # {'p':2, 'e':1, 'n':0}
        if noun_pn==0 and adj_pn==2:
            print(pair, utt)
            is_lack_common = True
            break
    
    if is_lack_common:
        y_pred.append(1)
    else:
        y_pred.append(0)
    # break
        


  2%|▏         | 28/1386 [00:01<00:32, 42.30it/s]

['熱中症', 'いい'] 熱中症はいいですね
['病院', '有名'] 病院は有名ですね


 14%|█▍        | 195/1386 [00:03<00:14, 84.66it/s]

['熱中症', 'いい'] 熱中症はいいですね


 21%|██        | 286/1386 [00:04<00:11, 92.53it/s]

...

 56%|█████▌    | 772/1386 [00:11<00:09, 63.66it/s]

['鉄拳', '好き'] サッカーチームのユベントスFCでは、薬師丸ひろ子と鉄拳が好きです。鉄拳選手はワールドカップでも素晴らしい活躍をしましたよね。
['葛藤', '見もの'] 薬師丸ひろ子さんと小泉今日子さんの心の葛藤が見ものでしたね。私もブラタモリ見ましたよ。


 75%|███████▍  | 1033/1386 [00:15<00:04, 84.89it/s]

['排気ガス', 'すごい'] 排気ガスすごいんですか！？知らなかったです。


100%|██████████| 1386/1386 [00:20<00:00, 69.06it/s]


In [13]:
score(y, y_pred)

confusion matrix = 
 [[1367   12]
 [   7    0]]
accuracy =  0.9862914862914863
precision =  0.0
recall =  0.0
f1 score =  0.0


- 完全勝利に近い

        confusion matrix = 
        [[1328   14]
        [   0    7]]
        accuracy =  0.9896219421793921
        precision =  0.3333333333333333
        recall =  1.0
        f1 score =  0.5

In [14]:
len(y_pred)

1386

In [15]:
for utt, p in zip(sys_utt, y):
    if p==1:
        adj_nouns = dependent_ADJ_NOUN(utt)
        print(utt, adj_nouns)

ようよう []
スイカで有名か？？ [['スイカ', '有名']]
ちゅっちゅ []
どっちでもいいや []
分かったからそう急かすな []
おはよん。 []
なんで？ []
